<a href="https://colab.research.google.com/github/rahulranjan612/Classification/blob/master/Assignment_11_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

transaction_train = pd.read_csv('/content/drive/My Drive/ML DataSets/train_transaction.csv')
transaction_train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20187,3007187,0,519594,45.00,W,15111,310.0,150.0,mastercard,224.0,debit,191.0,87.0,NaN,NaN,hotmail.com,NaN,8.0,9.0,0.0,0.0,4.0,8.0,0.0,0.0,5.0,0.0,5.0,0.0,59.0,5.0,486.0,486.0,7.0,486.0,7.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,170.0,0.0,0.0,170.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20188,3007188,0,519596,15.00,S,16435,298.0,150.0,visa,226.0,debit,337.0,87.0,NaN,NaN,NaN,gmail.com,2.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,4.0,2.0,64.0,64.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,...,0.0,0.0,1.0,2.0,1.0,1.0,0.0,60.0,40.0,0.0,40.0,0.0,20.0,0.0,0.0,0.0,0.0,20.0,20.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,40.0,40.0,0.0,20.0,20.0,0.0,0.0,0.0
20189,3007189,0,519597,92.00,W,7919,194.0,150.0,mastercard,202.0,debit,325.0,87.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,238.0,238.0,237.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190,3007190,0,519609,82.95,W,2803,100.0,150.0,visa,226.0,debit,299.0,87.0,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,4.0,2.0,88.0,88.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [2]:
transaction_train.info()
trans_corr = transaction_train.head(1000)
trans_corr.corr().isFraud.sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20192 entries, 0 to 20191
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 60.7+ MB


isFraud    1.000000
V199       0.378018
V196       0.370067
V114       0.362762
V122       0.362762
             ...   
V117            NaN
V118            NaN
V119            NaN
V241            NaN
V305            NaN
Name: isFraud, Length: 380, dtype: float64

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
transaction_train = transaction_train.fillna(0)
transaction_train = transaction_train.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
drop_columns = ['isFraud']
features = transaction_train.drop(drop_columns,axis=1)
labels = transaction_train[['isFraud']]

transaction_train.columns[transaction_train.isnull().any()].tolist()
features

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0,18831,1817,4,808,0,20,2,19,1,61,14,246,0,0,0,1,1,0,0,0,1,0,0,1,0,84,0,1,1,46,0,34,6,0,1,0,0,0,35,...,0,0,0,0,0,1,0,238,0,0,0,0,0,0,0,0,0,77,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,18832,899,4,1862,288,20,3,2,1,63,14,0,0,17,0,1,1,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,6,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,18833,1661,4,2231,369,20,4,29,2,65,14,380,0,36,0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,6,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,18834,1506,4,1638,442,20,3,5,2,100,14,0,0,52,0,101,154,0,0,0,103,0,0,1,0,1,0,107,1,16,16,0,504,0,1,0,0,0,496,...,0,0,0,0,0,1,869,852,1911,0,1210,0,93,0,0,0,470,174,812,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,18835,1506,1,2200,393,20,3,2,1,84,14,0,0,17,0,1,1,0,0,0,1,0,1,0,1,1,0,1,1,0,0,0,6,0,1,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20187,20187,18826,1390,4,1044,198,20,3,50,2,26,14,0,0,20,0,197,206,0,0,148,153,0,0,78,0,121,0,374,121,430,430,329,436,332,1,0,0,0,430,...,0,0,0,0,0,1,0,803,0,0,472,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20188,20188,18827,353,3,1306,186,20,4,51,2,70,14,0,0,0,15,101,132,0,15,0,0,0,39,0,1,1,0,187,82,480,477,0,6,0,1,0,410,0,0,...,0,0,1,3,1,1,0,2839,1191,0,1338,0,239,0,0,0,0,392,294,0,0,0,0,12,9,0,1,1,0,0,0,0,75,47,0,26,11,0,0,0
20189,20189,18828,2111,4,2843,85,20,3,44,2,63,14,0,0,0,0,1,1,0,0,0,0,0,0,0,0,84,0,1,1,155,155,144,6,0,1,0,0,0,154,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20190,20190,18829,2003,4,1875,1,20,4,51,2,55,14,0,0,0,0,101,97,0,0,1,1,0,0,49,0,84,0,187,82,506,503,310,6,0,1,0,0,0,499,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.impute import SimpleImputer

# Will use frequent strategy for below fields 
frequent_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
constant_imputer = SimpleImputer(strategy='constant',fill_value='OTHER')

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import set_config
# set_config(display='diagram')

num_attr = features.select_dtypes(exclude=[object]).columns.values
cat_attr = features.select_dtypes(include=[object]).columns.values

# num_attr = ['card2','card3','card5','addr1','addr2','dist1','dist']

# cat_attr = ['card4','card6']

num_pipleline = Pipeline([('frequent_imputer',frequent_imputer),
                         ('stand_scaler' ,StandardScaler())])

cat_pipeline = Pipeline([
                        ('one_hot_enc',OneHotEncoder())
])

pipeline = ColumnTransformer([('numeric_transformer',num_pipleline,num_attr),
                             ('cat_transformer',cat_pipeline,cat_attr)
                             ])


pipeline

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('numeric_transformer',
                                 Pipeline(memory=None,
                                          steps=[('frequent_imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='most_frequent',
                                                                verbose=0)),
                                                 ('stand_scaler',
                                                  StandardScaler(copy=True,
                                                                 with_

In [6]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc_labels = enc.fit_transform(labels.values.ravel())
enc_labels

array([0, 0, 0, ..., 0, 0, 0])

In [7]:
from sklearn.neighbors import KNeighborsClassifier
model_pipeline = Pipeline([
  ('processing', pipeline),
  ('model', KNeighborsClassifier())
])
model_pipeline

Pipeline(memory=None,
         steps=[('processing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric_transformer',
                                                  Pipeline(memory=None,
                                                           steps=[('frequent_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='most_frequent',
                               

In [8]:
model_pipeline.fit(features,labels.values.ravel())

Pipeline(memory=None,
         steps=[('processing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric_transformer',
                                                  Pipeline(memory=None,
                                                           steps=[('frequent_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='most_frequent',
                               

In [9]:
test_df = pd.read_csv('/content/drive/My Drive/ML DataSets/test_transaction.csv')
test_df
test_df = test_df.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
prediction = model_pipeline.predict(test_df)
prediction

array([0, 0, 0, ..., 1, 0, 0])

In [10]:
predictions_df = pd.DataFrame({
            'TransactionID': test_df.TransactionID,
            'isFraud': prediction
})

predictions_df.to_csv('/content/drive/My Drive/ML DataSets/predictions.csv',index=False)

In [11]:
# from sklearn.metrics import accuracy_score
# accuracy_score(test_df, model_pipeline.predict(features))

In [12]:
import joblib
joblib.dump(model_pipeline, 'fraudulent_model.pkl')
joblib.dump(prediction, 'fraudulent_prediction.pkl')
joblib.dump(test_df, 'test_df.pkl')

['test_df.pkl']

In [13]:
test_df

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0,0,885,4,94,10,23,4,41,1,7,6,1,255,16,28,173,164,0,0,166,97,0,0,107,0,118,1,20,121,356,356,173,332,186,360,201,917,24,355,...,0,0,0,0,0,0,0,2830,0,0,1875,0,0,0,0,0,0,0,0,0,0,0,24,65,46,3,27,12,19,47,35,102,204,147,19,108,51,62,89,72
1,1,1,1312,4,2318,10,23,4,41,1,29,6,416,255,2,28,142,107,0,0,0,1,0,0,67,0,1,1,25,79,56,56,420,595,522,360,201,917,24,148,...,0,0,0,0,0,0,0,1867,2023,0,1262,0,891,0,0,0,0,0,0,0,0,0,24,65,46,3,27,12,19,47,35,102,204,147,19,108,51,62,89,72
2,2,2,411,4,2353,459,23,4,41,1,57,6,320,255,19,28,117,107,0,0,0,105,0,0,87,0,82,0,115,79,43,43,2,723,2,360,201,917,24,42,...,0,0,0,0,0,0,0,502,0,0,73,0,0,0,0,0,0,0,0,0,351,0,24,65,46,3,27,12,19,47,35,102,204,147,19,108,51,62,89,72
3,3,2,797,4,217,252,23,4,24,1,16,6,177,255,16,28,162,107,0,0,1,1,0,0,67,0,82,0,476,102,357,357,286,160,304,360,201,917,24,160,...,0,0,0,0,0,0,0,0,0,0,0,0,0,269,577,363,0,0,0,0,0,0,24,65,46,3,27,12,19,47,35,102,204,147,19,108,51,62,89,72
4,4,3,1581,4,1778,343,23,2,2,1,24,6,522,255,16,28,173,164,0,0,107,105,0,0,96,0,131,0,51,121,135,135,0,135,0,360,201,917,24,135,...,0,0,0,0,0,0,1137,1061,1885,304,703,167,834,478,329,644,0,0,0,0,0,0,24,65,46,3,27,12,19,47,35,102,204,147,19,108,51,62,89,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22900,22900,22396,988,4,2045,401,23,4,41,1,4,6,228,255,32,28,142,143,0,0,107,88,0,0,67,0,118,0,175,90,584,553,420,726,556,360,201,917,24,413,...,0,0,0,0,0,0,0,2778,0,0,1252,0,0,0,0,0,0,0,0,0,201,0,24,65,46,3,27,12,19,47,35,102,204,147,19,108,51,62,89,72
22901,22901,22397,1834,4,2131,439,23,4,41,1,22,6,689,255,16,28,1,1,0,0,1,1,0,0,0,0,1,0,1,1,0,639,453,0,556,360,201,917,24,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,65,46,3,27,12,19,47,35,102,204,147,19,108,51,62,89,72
22902,22902,22398,312,2,1496,68,23,4,41,0,35,6,689,255,53,13,1,1,0,1,0,1,0,1,0,1,1,0,1,1,0,639,453,726,556,360,201,268,21,740,...,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22903,22903,22399,988,4,437,438,23,4,41,1,65,6,604,255,16,28,142,107,0,0,107,88,0,0,79,0,1,0,131,90,134,134,35,660,35,360,201,917,24,683,...,0,0,0,0,0,0,0,3997,0,0,2577,0,0,0,0,0,0,0,0,0,0,0,24,65,46,3,27,12,19,47,35,102,204,147,19,108,51,62,89,72
